In [1]:
import numpy as np
import cv2

import PIL.Image as Image

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub       # <------------------------
# search for tensorflow hub. here we can get various pretrained models

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
data_dir = '.\\useful_data\\datasets\\flower_photos_copy'

In [3]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir   

WindowsPath('useful_data/datasets/flower_photos_copy')

In [4]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [5]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [17]:
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224, 224))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [18]:
X = np.array(X)
y = np.array(y)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [20]:
X_train[0].shape

(224, 224, 3)

In [21]:
# preprocessing

X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [10]:
# ------------------  Now take pre-trained model and retrain it using flowers images  -------------------------------


feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4" 
# url where pretrained model is present  (without prediction layer)
# mobilenet V2 : pretrained CNN

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)  # trainable=False: layers are freezed

In [11]:
model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(5, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [26]:
model.compile(
  optimizer="adam",
  loss='sparse_categorical_crossentropy',
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
3/3 [==============================] - 5s 656ms/step - loss: 0.1942 - acc: 0.9701
Epoch 2/5
3/3 [==============================] - 2s 643ms/step - loss: 0.1421 - acc: 0.9701
Epoch 3/5
3/3 [==============================] - 2s 630ms/step - loss: 0.1246 - acc: 0.9851
Epoch 4/5
3/3 [==============================] - 2s 648ms/step - loss: 0.1034 - acc: 0.9851
Epoch 5/5
3/3 [==============================] - 3s 665ms/step - loss: 0.0859 - acc: 1.0000


In [27]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 2s 2s/step - loss: 0.7654 - acc: 0.6957


[0.7654185891151428, 0.695652186870575]

In [30]:
predicted = model.predict(X_test_scaled)
predicted_class = [np.argmax(arr) for arr in predicted]
predicted_class[:5]

[4, 0, 0, 2, 3]

In [31]:
y_test[:5]

array([0, 0, 2, 2, 3])